In [3]:
import pandas as pd
df = pd.read_csv("ipl_2021_matches.csv")
first_match = df.iloc[0]
df.head()

,year,series_type,series_name,match_no,match_type,match_name,match_href,match_team1,match_team2,match_datetime_start,match_date_end,match_venue
0,2021,T20,Indian Premier League 2021,NaN,League,"MUMBAI INDIANS vs ROYAL CHALLENGERS BENGALURU,...",https://www.cricbuzz.com/cricket-scores/35612/...,NaN,NaN,NaN,NaN,NaN
1,2021,T20,Indian Premier League 2021,NaN,League,"CHENNAI SUPER KINGS vs DELHI CAPITALS, 2nd Match",https://www.cricbuzz.com/cricket-scores/35617/...,NaN,NaN,NaN,NaN,NaN
2,2021,T20,Indian Premier League 2021,NaN,League,"SUNRISERS HYDERABAD vs KOLKATA KNIGHT RIDERS, ...",https://www.cricbuzz.com/cricket-scores/35618/...,NaN,NaN,NaN,NaN,NaN
3,2021,T20,Indian Premier League 2021,NaN,League,"RAJASTHAN ROYALS vs PUNJAB KINGS, 4th Match",https://www.cricbuzz.com/cricket-scores/35622/...,NaN,NaN,NaN,NaN,NaN
4,2021,T20,Indian Premier League 2021,NaN,League,"KOLKATA KNIGHT RIDERS vs MUMBAI INDIANS, 5th M...",https://www.cricbuzz.com/cricket-scores/35627/...,NaN,NaN,NaN,NaN,NaN


In [4]:
match_href = first_match['match_href'] 
match_id = match_href.split('/')[4]     
print("Match ID:", match_id)


Match ID: 35612


In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

driver = webdriver.Chrome()
match_id = '35612'

commentary_url = f"https://www.cricbuzz.com/live-cricket-commentary/{match_id}"
driver.get(commentary_url)
time.sleep(3)


for _ in range(10):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
    time.sleep(1)

commentary_blocks = driver.find_elements(By.CSS_SELECTOR, '.cb-col.cb-col-100.cb-col.cb-com-ln')
ball_data = []

for block in commentary_blocks:
    try:
        over = block.find_element(By.CSS_SELECTOR, '.cb-col.cb-col-8.text-bold').text.strip()
        text = block.find_element(By.CSS_SELECTOR, '.cb-col.cb-col-92.cb-com-ln').text.strip()
        ball_data.append({
            'year': 2021,
            'series_type': 'IPL',
            'series_name': 'Indian Premier League 2021',
            'match_no': '1st Match',
            'match_type': 'T20 IPL',
            'match_id': match_id,
            'match_winning_team': '',
            'match_tie_breaker': '',
            'match_toss': '',
            'ball_no': over.replace(" ", ""),
            'over_no': over,
            'ball_commentary': text
        })
    except:
        continue


scorecard_url = f"https://www.cricbuzz.com/live-cricket-scorecard/{match_id}"
driver.get(scorecard_url)
time.sleep(3)

scorecard_data = {}


try:
    match_status = driver.find_element(By.CSS_SELECTOR, ".cb-text-live, .cb-text-complete, .cb-text-abandon").text
    scorecard_data['match_status'] = match_status
except:
    scorecard_data['match_status'] = 'NA'

info_blocks = driver.find_elements(By.CSS_SELECTOR, '.cb-col.cb-col-100.cb-mtch-info-itm')
for block in info_blocks:
    text = block.text
    if "toss" in text.lower():
        scorecard_data['match_toss'] = text
    elif "won" in text.lower():
        scorecard_data['match_winning_team'] = text
    elif "super over" in text.lower() or "tie" in text.lower():
        scorecard_data['match_tie_breaker'] = text
    elif "umpire" in text.lower():
        if "third" in text.lower():
            scorecard_data['third_umpires'] = text
        else:
            scorecard_data['umpires'] = text
    elif "referee" in text.lower():
        scorecard_data['match_referee'] = text
    elif ":" in text:
        scorecard_data['match_datetime'] = text


team_cards = driver.find_elements(By.CSS_SELECTOR, '.cb-col.cb-col-100.cb-scrd-hdr-rw')

team_names, team_scores, team_wickets = [], [], []
for team_card in team_cards:
    name = team_card.text.split('Innings')[0].strip()
    team_names.append(name)
    try:
        score = team_card.find_element(By.XPATH, "../div[2]").text.strip()
        sc, wk = score.split('/') if '/' in score else (score, '10')
        team_scores.append(sc.strip())
        team_wickets.append(wk.strip())
    except:
        team_scores.append('NA')
        team_wickets.append('NA')


try:
    all_blocks = driver.find_elements(By.CSS_SELECTOR, '.cb-col.cb-col-100.cb-ltst-wgt-hdr')
    squads = [b.text.lower() for b in all_blocks]
    squad_data = [b.find_element(By.XPATH, "following-sibling::div").text for b in all_blocks]
except:
    squads = []
    squad_data = []

get_squad_info = lambda title: squad_data[squads.index(title)] if title in squads else 'NA'


scorecard_data.update({
    'team1_name': team_names[0] if len(team_names) > 0 else 'NA',
    'team2_name': team_names[1] if len(team_names) > 1 else 'NA',
    'team1_score': team_scores[0] if len(team_scores) > 0 else 'NA',
    'team1_wickets': team_wickets[0] if len(team_wickets) > 0 else 'NA',
    'team2_score': team_scores[1] if len(team_scores) > 1 else 'NA',
    'team2_wickets': team_wickets[1] if len(team_wickets) > 1 else 'NA',
    'team1_captain': get_squad_info('captain'),
    'team1_players': get_squad_info('players'),
    'team1_bench': get_squad_info('bench'),
    'team1_support_staff': get_squad_info('support staff'),
    'team2_captain': get_squad_info('captain'),
    'team2_players': get_squad_info('players'),
    'team2_bench': get_squad_info('bench'),
    'team2_support_staff': get_squad_info('support staff')
})

for row in ball_data:
    row.update(scorecard_data)

final_df = pd.DataFrame(ball_data)
final_df.to_csv("first_match_full_data.csv", index=False)

print("✅ Done! File saved as 'first_match_full_data.csv'")
driver.quit()



✅ Done! File saved as 'first_match_full_data.csv'
